# Parameterizable behavior trees

This example notebook demonstrates how to create parameterizable behavior trees (PBTs).
It builds on the sequential pick-and-place process from a previous example.

In particular, this example covers:

- Creating parameterizable behavior trees
    - Creating tree parameters
    - Using tree parameters as skill parameters
- Sideloading and using parameterizable behavior trees

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [portal.intrinsic.ai](https://portal.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `pick_and_place:pick_and_place_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `pick_and_place:pick_and_place_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

First, connect and define convenience shortcuts:

In [ ]:
from intrinsic.math.python import data_types
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt

solution = deployments.connect_to_selected_solution()

executive = solution.executive
resources = solution.resources
skills = solution.skills
world = solution.world
simulator = solution.simulator
proto_builder = solution.proto_builder

attach_object_to_robot = skills.ai.intrinsic.attach_object_to_robot
control_suction_gripper = skills.ai.intrinsic.control_suction_gripper
detach_object = skills.ai.intrinsic.detach_object
enable_gripper = skills.ai.intrinsic.enable_gripper
move_robot = skills.ai.intrinsic.move_robot
plan_grasp = skills.ai.intrinsic.plan_grasp

Define generic skills and create an initialization tree:

In [ ]:
init_gripper = enable_gripper(
    clear_faults=True, gripper=resources.picobot_gripper)

move_home = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.home,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ],
    arm_part=world.robot,
)

# Turn on the suction gripper which will "attach" the building block in the
# simulation/real world.
grasp = control_suction_gripper(
    grasp=control_suction_gripper.intrinsic_proto.eoat.GraspRequest()
)

# Turn off the suction gripper which will "drop" the building block in the
# simulation/real world.
release = control_suction_gripper(
    release=control_suction_gripper.intrinsic_proto.eoat.ReleaseRequest()
)

In [ ]:
initialize = bt.SubTree(
    name="Initialize",
    behavior_tree=bt.Sequence(
        [
            bt.Task(action=init_gripper, name="Enable gripper"),
            bt.Task(action=move_home, name="Move home"),
        ]
    ),
)

Building a parameterizable behavior tree makes it necessary to define the parameters. You can pick python types like `int`, `float`, `str`, `bool` or well known types.
Get a list of the well known types using the following call.

In [ ]:
proto_builder.get_well_known_types()

In this example, the goal is to build a generic tree that grasps an object. Thus create its `GraspParams` message that takes the `object` to grasp as its only parameter.

In [ ]:
grasp_param_message = proto_builder.create_message("intrinsic_proto.grasp_tree", "GraspParams", {"object" : "intrinsic_proto.world.ObjectReference"})

Create a parameterizable behavior tree named `grasp_tree`. This is just a `BehaviorTree` instance that is initialized as a parameterizable tree.

The `skill_id` make this parameterizable tree re-usable in a process like a skill. In addition to that the `grasp_param_message` defined above defines what parameters the tree has.

In [ ]:
grasp_pbt = bt.BehaviorTree(name="grasp_tree")
grasp_pbt.initialize_pbt_with_protos(
    skill_id = "ai.intrinsic.grasp_tree", display_name="Grasp Tree",
    parameter_proto = grasp_param_message)

Create the skill calls for the `grasp_pbt` as for any other tree.

Note that the parameters of the behavior tree are available at `grasp_pbt.params.<param>`. These can be used in the same way that one would use a skill return value.

In [ ]:
# Moves the global frames `world.grasp_frame` and `world.pregrasp_frame` to
# suitable grasp/pregrasp positions above the building block.
#
# Note that the objects refer to the `object` parameter of the grasp_pbt.
# This makes plan_grasp plan grasps for whatever object is passed in when using
# this parameterizable behavior tree.
plan_block_grasp = plan_grasp(
    objects=[grasp_pbt.params.object],
    tool_frame=world.picobot_gripper.tool_frame,
    advanced_params=(
        plan_grasp.intrinsic_proto.manipulation.skills.PlanGraspAdvancedParams(
            product_part_name="building_block",
            output_pregrasp_frame=world.pregrasp_frame,
            output_grasp_frame=world.grasp_frame,
        )
    ),
)

move_pregrasp = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.pregrasp_frame,
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ],
    arm_part=world.robot,
)

collisions_disabled = move_robot.intrinsic_proto.world.CollisionSettings(
    disable_collision_checking=True
)
move_grasp_unsafe = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.grasp_frame,
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.LINEAR,
            collision_settings=collisions_disabled,
        )
    ],
    arm_part=world.robot,
)

# Note that the attach_object_to_robot skill uses the `object` input parameter passed in from the grasp_pbt
attach_block = attach_object_to_robot(
    gripper_entity=world.picobot_gripper, object_entity=grasp_pbt.params.object
)

move_up_unsafe = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.picobot_gripper.tool_frame,
                target_frame_offset=data_types.Pose3(
                    translation=[0, 0, -0.03]),
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.LINEAR,
            collision_settings=collisions_disabled,
        )
    ],
    arm_part=world.robot,
)

grasp_pbt.set_root(bt.Sequence([
    plan_block_grasp,
    move_pregrasp,
    move_grasp_unsafe,
    grasp,
    attach_block,
    move_up_unsafe]))

Likewise create a tree to place an object. Here also define a `place_frame` to determine where the object should be placed.

In [ ]:
place_param_message = proto_builder.create_message("intrinsic_proto.place_tree", "PlaceParams",
                                                   {"object" : "intrinsic_proto.world.ObjectReference",
                                                    "place_frame" : "intrinsic_proto.world.TransformNodeReference"})

As for `grasp_pbt` create the `place_pbt` as a `BehaviorTree` and initialize it to be a parameterizable behavior tree by assigning a `skill_id` and defining `place_param_message` to be its parameters.

In [ ]:
place_pbt = bt.BehaviorTree(name="place_tree")
place_pbt.initialize_pbt_with_protos(
    skill_id = "ai.intrinsic.place_tree", display_name="Place Tree",
    parameter_proto = place_param_message)

Create the skill calls for placing `object` at the `place_frame`.

In [ ]:
# Note that the move_drop skill's target_frame is the `place_frame` of the `place_pbt`'s params.
move_drop = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=place_pbt.params.place_frame,
                target_frame_offset=data_types.Pose3(
                    rotation=data_types.Rotation3.from_euler_angles(
                        rpy_degrees=[180, 0, 90]
                    ),
                    translation=[0, 0, 0.05],
                ),
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ],
    arm_part=world.robot,
)

release = control_suction_gripper(
    release=control_suction_gripper.intrinsic_proto.eoat.ReleaseRequest()
)

# Detach the `object` given as an input parameter in the `place_pbt`.
detach = detach_object(
    gripper_entity=world.picobot_gripper, object_entity=place_pbt.params.object
)

place_pbt.set_root(bt.Sequence([
    move_drop,
    release,
    detach,
    move_up_unsafe,
    move_home]))

Sideload the created trees. Afterwards these will be available as new skills with the given parameters.

Thus call `update_skills()` here to refresh the list of skills. If you are using the frontend to show the process then reload the frontend, so that it can read the new skill definitions.

In [ ]:
solution.pbt_registry.sideload_behavior_tree(grasp_pbt)
solution.pbt_registry.sideload_behavior_tree(place_pbt)
solution.update_skills()

Build a process tree that moves `building_block0` to `target_right` by using the grasp and place trees.

In [ ]:
executive.run(bt.Sequence(children=[initialize,
                                    bt.Task(name="Grasp block0", action=skills.ai.intrinsic.grasp_tree(object=world.building_block0)),
                                    bt.Task(name="Place block0", action=skills.ai.intrinsic.place_tree(object=world.building_block0, place_frame=world.target_right))]))

Like skills we can re-use our parameterizable behavior trees with other parameters. The following tree moves the object back to `target_left`.

In [ ]:
executive.run(bt.Sequence(children=[initialize,
                                    bt.Task(name="Grasp block0", action=skills.ai.intrinsic.grasp_tree(object=world.building_block0)),
                                    bt.Task(name="Place block0", action=skills.ai.intrinsic.place_tree(object=world.building_block0, place_frame=world.target_left))]))

This notebook showed how to create two simple to use parameterizable behavior trees that make it possible to grasp and place an object.
To this end the necessary skills are contained in the parameterizable trees and appear as skills to be used in a process tree.

You can now experiment with these skills, e.g., try moving a different object or multiple locations.

Consider wrapping re-occurring functionalities in your processes in a parameterizable behavior tree. This prevents re-doing the same pattern multiple times and any improvements to a parameterizable behavior tree immediately apply to all call sites.